In [11]:
reset -fs

# Install Packages

In [1]:
%pylab inline
import networkx as nx
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import numpy as np
import itertools
import plotly.tools as tls
import pickle
import yaml
import os
credentials = yaml.load(open(os.path.expanduser('plotlyCred.yml')))
plotly.tools.set_credentials_file(credentials.get('username'), api_key=credentials.get('api_key'))

Populating the interactive namespace from numpy and matplotlib


In [6]:
# tls.get_embed('https://plot.ly/~chaudao19/61')

# Cleaning Data

In [14]:
df = pd.read_csv("techtopics.csv")
df.columns = ["index","topics"]

In [19]:
def clean_df(df):
    indice = df['topics'].apply(lambda x: type(x) == str)
    df = df[indice]
    df['key'] = df['topics'].apply(lambda x: x[1:-1].split(','))
    df['key'] = df['key'].apply(lambda x: [word.strip(" ") for word in x])
    df = df.reset_index()
    df = pd.DataFrame(df['key'])
    return df

In [20]:
df = clean_df(df);

In [21]:
df.head(5)

,key
0,"[u'Web Standards', u'PHP', u'Open Source', u'W..."
1,"[u'Social Software', u'New Technology', u'Web ..."
2,"[u'PHP', u'Open Source', u'Web Design', u'MySQ..."
3,[u'MySQL']
4,"[u'Enterprise Architecture', u'JavaScript', u'..."


In [22]:
def key_list(df):
    urlkey_unique = df['key'].tolist()
    merged_list = list(itertools.chain(*urlkey_unique))
    return merged_list
def gettuples(df,column):
    urlkey_unique = df[column].tolist()
    merged_list = list(itertools.chain(*urlkey_unique))
    return merged_list

# Fitting Data to Plotly Graph

In [23]:
#Initialize the graph
G= nx.Graph()

In [24]:
# Nodes: Unique entity (listofuniqueitmes)
# Edges: Connection between nodes (listoftuplesbetween2nodes)
df['edges'] = df['key'].apply(lambda x: [(a,b) for a in x for b in x if a != b])
nodes = list(set(key_list(df[:100])))
G.add_nodes_from(nodes)
edges = gettuples(df[:100],"edges")
G.add_edges_from(edges)

In [25]:
# plt.figure(figsize=(40,30))
# nx.draw(G,node_color='c',edge_color='k', with_labels=True);

In [26]:
# pos: assign x,y coordinates for each node
pos= nx.spring_layout(G)
G.node = pos
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(G,ncenter)

In [27]:
edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
for edge in G.edges():
    x0, y0 = G.node[edge[0]]
    x1, y1 = G.node[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]
node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[], 
        size=10,         
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    node_trace['text'].append(node)

In [28]:
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['color'].append(len(adjacencies))
    node_info = '# of connections: '+str(len(adjacencies))
    node_trace['text'].append(node_info)

In [31]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br> Tech Topics - Network Graph',
                titlefont=dict(size=16),
                showlegend=True, 
                width=800,
                height=650,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a Chau Dao </a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=True),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=True)))

py.iplot(fig, filename='TechTopics')